In [155]:
import faker
import json
import random
faker = faker.Faker()

In [156]:
albumNames = ['Bitten bullet','Parliment of fools','Fluke','Here we go','Grains of sand','Dark matters',
'Criss cross','Balls to the wall','Eye for an eye','Code word', 'Grains of sand','Dreams and nightmares','Around the bush','Crazy eyes',
'Form over function','Speak of the god','Big bad wolf','Deaths dead','Final breath','Rocket science','Curtain fall',
'Speak of the god','Zero experience','Beggars and thieves','Hand in hand','Fight and flight','Rocket surgery','Gods advocate',
'Crocodile tears','Doctor in the house', 'X marks the spot','New dimension','Company of two','Flower shower',
'The drawing board','For eternity','Army of ants','Chip off the old block','Expert opinion','Speak of the demon',
'Apparatus','Tongue and cheeks','Ghost stories','Brain teaser','Devils advocate','Ask and receive','Beads of sweat',
'Perfect storm','Curiosity','Childhood memoires']
print(len(albumNames))

50


In [157]:
bandNames = ['Suave','Vanished Generation','Shallow Accident','Cinder','Realm','Bittersweet',
'Bravado','Format of Fear','Monster of Drama','Switch of the Underground',
'Cinematic Gossip','Simple Nemesis','Daydream Diary','Dilemma','Alien','Cataclysm','Wraith','Torment of Wit','Mirror of Momentum',
'Brotherhood of Blitz']
print(len(bandNames))

20


In [158]:
genres = ['Pop', 'Rock', 'Jazz', 'Classical', 'Hip Hop', 'Country', 'Reggae', 'Blues' , 'Folk']

In [159]:
def generateIsNew() :
    x = random.randint(1,20)
    if x < 2:
        return True
    else:
        return False

In [160]:
random_records = []
startingId = 100
isNewCount =0

for i in range(100):

    
    #img_number = random.randint(1, 75)
    random_record = {
        'id': startingId + i,
        'band_name' : random.choice(bandNames),
        'album_name': random.choice(albumNames),
        'genre': random.choice(genres),
        'release_date' : faker.date(),
        'record_label' : faker.company(),
        'duration' : f"{random.randint(30,90)}:{random.randint(0,59):02d}",
        'price' : f"{random.randint(15,25)}.00",
        'numSold' : random.randint(20,100),
        'is_new' : generateIsNew()
        
    }
    random_records.append(random_record)
    # if random_record.is_new == true:
    #      isNewCount += 1
    
#display (random_records)
# display(isNewCount)

In [161]:
file_name = 'records.json'
with open(file_name, 'w') as fout:
    fout.write(json.dumps(random_records, indent=2))

In [162]:
def generateIsBought():
    x = random.randint(0,100)
        
    

In [163]:
dummy_data = []
startingId = 1000

for i in range(1000):

    
    #img_number = random.randint(1, 75)
    cart = {
        'id': startingId + i,
        'band_name' : random.choice(bandNames),
        'album_name': random.choice(albumNames),
        'genre': random.choice(genres),
        'release_date' : faker.date(),
        'record_label' : faker.company(),
        'duration' : f"{random.randint(30,90)}:{random.randint(0,59):02d}",
        'price' : f"{random.randint(15,25)}.00",
        'num_sold' : random.randint(20,100),
        'criticRating' : random.randint(1,5),
        'next_album_to_buy':random.choice(albumNames)

        
        
    }
    dummy_data.append(cart)
    # if random_record.is_new == true:
    #      isNewCount += 1
    
#display (dummy_data)
# display(isNewCount)

In [164]:
import pandas as pd

df = pd.DataFrame(dummy_data)
                  
df.to_csv('cart_product_data.csv', index = False)
#display (df)




In [165]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score


data = pd.read_csv('cart_product_data.csv')

# Define features (X) and target variable (y)
X = data[['genre', 'band_name', 'num_sold']]
y = data['next_album_to_buy']



# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of train and test sets
print("train_test_split:")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


train_test_split:
X_train shape: (800, 3)
X_test shape: (200, 3)
y_train shape: (800,)
y_test shape: (200,)


In [166]:
# Initialize the KNN classifier
k = 5  # Number of neighbors
knn = KNeighborsClassifier(n_neighbors=k)

In [167]:
import pickle
# Define categorical features for one-hot encoding
categorical_features = ['genre', 'band_name', 'num_sold']

# Apply one-hot encoding to categorical features

#we can pickle the processor and use again in flask
    
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

file_path = 'preprocessor.pkl'



In [168]:
# Fit and transform the training data
X_train_encoded = preprocessor.fit_transform(X_train)

# Transform the testing data
X_test_encoded = preprocessor.transform(X_test)

In [169]:
# Train the classifier
knn.fit(X_train_encoded, y_train)

KNeighborsClassifier()

In [170]:
# Predict the cities for the test set
y_pred = knn.predict(X_test_encoded)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.035


In [171]:
next_album = pd.DataFrame({
    'genre': ['Jazz'],
    'band_name' : ['Dilemma'],
    'num_sold': [45]
    
})
#display(next_album)

# Transform the new customer's data using the preprocessor
next_album_encoded = preprocessor.transform(next_album)

predicted = knn.predict(next_album_encoded)
print("Reccomended album for you is: ", predicted[0])

Reccomended album for you is:  Flower shower


In [172]:
import pickle

# Specify the file path to save the pipeline
file_path = 'model_pipeline.pkl'

# Save the pipeline to disk
with open(file_path, 'wb') as file:
    pickle.dump(knn, file)

In [173]:
# Save the pipeline to disk
file_path = 'preprocessor.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(preprocessor, file)